# Xử lý dữ liệu cho mô hình

### Product Feature

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
product = pd.read_csv(r'D:\Document\doan2\product_features.csv')

In [ ]:
product = product.drop_duplicates()
product.isnull().sum()
index_to_drop = product[product['brand_id'] == 0].index
product = product.drop(index_to_drop)
product = product.dropna(subset=['product_id'])
product['product_id'] = product['product_id'].astype(int)

In [ ]:
import statsmodels.api as sm

X = product[['price', 'rating_average', 'review_count', 'brand_id', 'seller_id', 'total_follower', 'rating_seller']]
y = product['quantity']

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X, y)

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure()
plt.title("Feature Importances")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), [X.columns[i] for i in indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()


In [ ]:
correlation_matrix = product[['review_count', 'price', 'brand_id', 'rating_average', 'total_follower', 'seller_id', 'breadcrumbs_id']].corr()

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Correlation Matrix')
plt.show()

### User feature

In [ ]:
customer = pd.read_csv('D:/Document/doan2/data-doan2-clean/customer_clean.csv')
customer = customer[['created_by_id']]

### Rating

In [ ]:
rating = pd.read_csv('D:/Document/doan2/rating.csv')
rating = rating.drop_duplicates()
rating.isnull().sum()
rating = rating.dropna(subset=['review_date', 'review_title'])
rating = rating.drop(columns=['review_title', 'review_date'])

# Mô hình Content-Based Recommendation

In [ ]:
df_product = product.copy()
df_rating = rating.copy()
df_customer = customer.copy()

In [ ]:
df_product_encoded = pd.get_dummies(df_product, columns=['last_breadcrumbs_id', 'brand_id', 'seller_id'], drop_first=True)
features = df_product_encoded[['price', 'rating_average', 'review_count', 'total_follower'] + list(df_product_encoded.columns[df_product_encoded.columns.str.startswith('last_breadcrumbs_id_')]) + list(df_product_encoded.columns[df_product_encoded.columns.str.startswith('brand_id_')]) + list(df_product_encoded.columns[df_product_encoded.columns.str.startswith('seller_id_')])]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [ ]:
import joblib
import pandas as pd
features_scaled = joblib.load('D:/Document/doan2/features_scaled.pkl')
df_product = pd.read_pickle('D:/Document/doan2/df_product.pkl')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_single_product_similarity(product_id):
    if product_id not in df_product['product_id'].values:
        return "Product ID not found."
    
    product_index = df_product.index[df_product['product_id'] == product_id].tolist()[0]

    similarity_scores = cosine_similarity(features_scaled[product_index].reshape(1, -1), features_scaled)

    return pd.Series(similarity_scores.flatten(), index=df_product['product_id'])

def get_product_recommendations(product_id, num_recommendations=5):
    similarity_scores = get_single_product_similarity(product_id)
    
    recommended_products = similarity_scores.sort_values(ascending=False).index[1:num_recommendations + 1]
    
    return df_product[df_product['product_id'].isin(recommended_products)]


In [ ]:
recommended_products = get_product_recommendations(product_id=177822630, num_recommendations=30)

# Mô hình Collaborative Filtering Recommendation 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Lấy unique user_id và product_id để gán các chỉ số embedding
user_ids = df_rating['customer_id'].unique().tolist()
product_ids = df_rating['product_id'].unique().tolist()

# Tạo index mapping cho user_id và product_id
user_id_mapping = {id: idx for idx, id in enumerate(user_ids)}
product_id_mapping = {id: idx for idx, id in enumerate(product_ids)}

# Áp dụng mapping để chuẩn hóa ID
df_rating['user_idx'] = df_rating['customer_id'].map(user_id_mapping)
df_rating['product_idx'] = df_rating['product_id'].map(product_id_mapping)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train, test = train_test_split(df_rating, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model

# Thông số mô hình
n_users = len(user_ids)      # Số lượng người dùng
n_products = len(product_ids) # Số lượng sản phẩm
embedding_size = 50          # Kích thước vector embedding

# Input cho người dùng và sản phẩm
user_input = Input(shape=(1,), name='user_input')
product_input = Input(shape=(1,), name='product_input')

# Embedding cho người dùng và sản phẩm
user_embedding = Embedding(input_dim=n_users, output_dim=embedding_size, name='user_embedding')(user_input)
product_embedding = Embedding(input_dim=n_products, output_dim=embedding_size, name='product_embedding')(product_input)

# Flatten để biến embedding thành vector
user_vector = Flatten()(user_embedding)
product_vector = Flatten()(product_embedding)

# Kết hợp vector người dùng và sản phẩm
concat = Concatenate()([user_vector, product_vector])

# Thêm các lớp Dense để tạo mạng neural
dense = Dense(128, activation='relu')(concat)
dense = Dropout(0.5)(dense)
dense = Dense(64, activation='relu')(dense)
dense = Dropout(0.5)(dense)
dense = Dense(32, activation='relu')(dense)

# Đầu ra là điểm đánh giá (hoặc xác suất)
output = Dense(1, activation='linear')(dense)

# Khởi tạo mô hình
model = Model(inputs=[user_input, product_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


In [ ]:
# Chuẩn bị dữ liệu đầu vào
train_user = train['user_idx'].values
train_product = train['product_idx'].values
train_ratings = train['review_rating'].values

test_user = test['user_idx'].values
test_product = test['product_idx'].values
test_ratings = test['review_rating'].values

# Huấn luyện mô hình
history = model.fit(
    [train_user, train_product],
    train_ratings,
    validation_data=([test_user, test_product], test_ratings),
    epochs=10,
    batch_size=256
)

In [ ]:
from tensorflow.keras.models import clone_model
new_model = clone_model(model)
new_model.set_weights(model.get_weights())
new_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model('D:/Document/doan2/model.h5')

In [ ]:
import pickle
with open('D:/Document/doan2/user_id_mapping.pkl', 'wb') as f_user:
    pickle.dump(user_id_mapping, f_user)
with open('D:/Document/doan2/product_id_mapping.pkl', 'wb') as f_product:
    pickle.dump(product_id_mapping, f_product)

In [ ]:
# Đánh giá trên tập kiểm tra
test_loss, test_mae = model.evaluate([test_user, test_product], test_ratings)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

In [ ]:
import numpy as np

user_id = 589097
product_id = 207374084

# Lấy index của user_id và product_id từ ánh xạ
user_idx = user_id_mapping.get(user_id)
product_idx = product_id_mapping.get(product_id)

# Kiểm tra xem user_idx và product_idx có hợp lệ không
if user_idx is None or product_idx is None:
    print("User ID hoặc Product ID không tồn tại trong ánh xạ.")
else:
    user_input = np.array([[user_idx]])     # Đầu vào cho user
    product_input = np.array([[product_idx]])  # Đầu vào cho product

    # Dự đoán rating
    predicted_rating = model.predict([user_input, product_input])

    # In kết quả dự đoán
    print(f"Predicted Rating: {predicted_rating[0][0]}")


# Luật kết hợp

In [ ]:
rating_df = pd.read_csv('D:/Document/doan2/rating.csv')
rating_df = rating_df.drop_duplicates()
rating_df = rating_df.dropna(subset=['review_date', 'review_title'])
rating_df = rating_df.drop(columns=['review_title'])
rating_df['review_date'] = pd.to_datetime(rating_df['review_date'])

In [ ]:
grouped_df = rating_df.groupby(['review_date', 'customer_id'])['product_id'].agg(lambda x: ', '.join(map(str, x))).reset_index()
grouped_df.rename(columns={'product_id': 'purchased_products'}, inplace=True)
filtered_df = grouped_df[grouped_df['purchased_products'].str.count(',') > 0]
filtered_df = filtered_df[['purchased_products']]
filtered = filtered_df.copy()

+ Kiểm tra sự tồn tại của luật kết hợp

In [ ]:
# Kiểm tra tần suất của từng sản phẩm
product_counts = pd.Series([item for transaction in transactions for item in transaction]).value_counts()
print(product_counts.head(10))
# Kiểm tra các sản phẩm có tần suất trên ngưỡng hỗ trợ
min_support_count = len(transactions) * 0.001
common_products = product_counts[product_counts >= min_support_count]
print("Số sản phẩm phổ biến đạt ngưỡng:", len(common_products))

In [ ]:
# Tính số sản phẩm trong mỗi giao dịch
transaction_lengths = [len(transaction) for transaction in transactions]
# Vẽ biểu đồ phân phối số lượng sản phẩm trong giao dịch
plt.figure(figsize=(8, 4))
plt.hist(transaction_lengths, bins=50, color='lightgreen', edgecolor='black')
plt.title("Phân phối số lượng sản phẩm trong mỗi giao dịch")
plt.xlabel("Số sản phẩm trong giao dịch")
plt.ylabel("Số lượng giao dịch")
plt.show()

In [ ]:
from collections import Counter
from itertools import combinations
# Đếm số lần xuất hiện của từng cặp sản phẩm
pair_counts = Counter()
for transaction in transactions:
    if len(transaction) > 1:
        pair_counts.update(combinations(transaction, 2))


In [ ]:
# Lấy top 20 cặp sản phẩm phổ biến
common_pairs = pd.DataFrame(pair_counts.most_common(20), columns=['Cặp sản phẩm', 'Tần suất'])
# Vẽ biểu đồ thanh cho top 20 cặp sản phẩm
plt.figure(figsize=(8, 4))
plt.barh(common_pairs['Cặp sản phẩm'].astype(str), common_pairs['Tần suất'], color='coral')
plt.title("Top 20 cặp sản phẩm phổ biến nhất")
plt.xlabel("Tần suất")
plt.ylabel("Cặp sản phẩm")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Tính ngưỡng hỗ trợ dưới dạng số lần xuất hiện
min_support_counts = [5, 10, 20, 50, 100]
num_products_by_support = [sum(product_counts >= threshold) for threshold in min_support_counts]
# Vẽ biểu đồ đường
plt.figure(figsize=(8, 4))
plt.plot(min_support_counts, num_products_by_support, marker='o', color='b')
plt.title("Số lượng sản phẩm đạt ngưỡng hỗ trợ khác nhau")
plt.xlabel("Ngưỡng hỗ trợ (số lần xuất hiện)")
plt.ylabel("Số lượng sản phẩm")
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns
# Lấy danh sách các sản phẩm phổ biến
top_products = product_counts.head(10).index
# Tạo ma trận đồng xuất hiện
co_occurrence = pd.DataFrame(0, index=top_products, columns=top_products)
for transaction in transactions:
    for product_a, product_b in combinations(transaction, 2):
        if product_a in top_products and product_b in top_products:
            co_occurrence.loc[product_a, product_b] += 1
            co_occurrence.loc[product_b, product_a] += 1
# Vẽ heatmap cho ma trận đồng xuất hiện
plt.figure(figsize=(8, 6))
sns.heatmap(co_occurrence, annot=True, cmap="YlGnBu", fmt="d")
plt.title("Ma trận đồng xuất hiện của các sản phẩm phổ biến")
plt.show()

### Xây dựng mô hình

In [ ]:
filtered['purchased_products'] = filtered['purchased_products'].apply(lambda x: x.split(','))
transactions = filtered['purchased_products'].tolist()
df = pd.DataFrame(transactions)

In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
filtered_transactions = [trans for trans in transactions if len(trans) > 1]
frequent_products = pd.Series([item for sublist in filtered_transactions for item in sublist]).value_counts()
common_products = set(frequent_products[frequent_products >= 10].index)
filtered_transactions = [[item for item in trans if item in common_products] for trans in filtered_transactions]
te = TransactionEncoder()
te_ary = te.fit(filtered_transactions).transform(filtered_transactions)
sparse_df = pd.DataFrame.sparse.from_spmatrix(csr_matrix(te_ary), columns=te.columns_)
frequent_itemsets = apriori(sparse_df, min_support=0.003, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

In [ ]:
import numpy as np
# Giả sử transactions là một danh sách các giao dịch
num_parts = 10  # Số phần bạn muốn chia
part_size = len(transactions) // num_parts
# Tạo danh sách các phần
parts = [transactions[i * part_size:(i + 1) * part_size] for i in range(num_parts)]

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
results = []  # Danh sách để lưu trữ kết quả cho từng phần
for part in parts:
    # Chuyển đổi phần giao dịch thành định dạng mà Apriori có thể sử dụng
    te = TransactionEncoder()
    te_ary = te.fit(part).transform(part)
    sparse_df = pd.DataFrame.sparse.from_spmatrix(csr_matrix(te_ary), columns=te.columns_)
    # Áp dụng thuật toán Apriori
    frequent_itemsets = apriori(sparse_df, min_support=0.003, use_colnames=True)
    results.append(frequent_itemsets)
# Kết hợp tất cả các itemsets lại với nhau
combined_results = pd.concat(results)

In [ ]:
# Nhóm theo các sản phẩm và tính tổng support
final_frequent_itemsets = combined_results.groupby('itemsets').sum().reset_index()
final_frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)

In [ ]:
rules = association_rules(final_frequent_itemsets, metric="confidence", min_threshold=0.1)
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]